In [1]:
import re
import ast
import pyprind
import requests
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

In [2]:
url = r'https://www.cardmarket.com/de/Magic'

In [3]:
n = 2

cards = []
headers = {'User-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/29.0.1547.76 Safari/537.36', 'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8','Accept-Encoding':'gzip,deflate,sdch'}
bar = pyprind.ProgPercent(n)
for i in range(n):
    bs = BeautifulSoup(requests.get(url+'/Products/Singles?resultsPage={}'.format(i), headers=headers).text, 'lxml')
    cards.append([tr.findAll('a')[1]['href'] for tr in bs.findAll('tr') if tr.find('a')])
    bar.update()

[100 %] Time elapsed: 00:00:01 | ETA: 00:00:00
Total time elapsed: 00:00:01


In [4]:
cards = {card for liste in cards for card in liste}
len(cards)

60

In [5]:
data = []
bar = pyprind.ProgPercent(len(cards))
for card in cards:
    html = requests.get('https://www.cardmarket.com'+card, headers=headers).text
    bs = BeautifulSoup(html, 'lxml')
    
    try:
        chartData = ast.literal_eval(re.search('chartData =(.*?);', str(bs)).group(1))
        df = pd.DataFrame(list(zip(chartData['labels'], chartData['datasets'][0]['data'])), columns=['Date', 'Price'])
        df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
        df['Link'] = card
        splitted = card.split('/')
        df['Name'] = splitted[6]
        df['Edition'] = splitted[5]
        df['Rarity'] = bs.find('tbody').find('td', {'class':'outerTop outerRight col_Odd col_1 cell_0_1'}).find('span')['onmouseover'].split("'")[1]
        df['Available'] = int(bs.find('span', {'itemprop':'offerCount'}).get_text())
        data.append(df)
    except:
        pass
    bar.update()

[100 %] Time elapsed: 00:01:08 | ETA: 00:00:00
Total time elapsed: 00:01:08


In [6]:
df = pd.concat(data)
df = df.set_index('Date')
df.to_csv('build/MGT_Cards.csv')
print(df.shape)
df.head()

(1646, 6)


,Price,Link,Name,Edition,Rarity,Available
Date,,,,,,
2018-03-26,0.51,/de/Magic/Products/Singles/Masters+25/Gedanken...,Gedankenwirbel,Masters+25,Common,4101
2018-03-27,0.52,/de/Magic/Products/Singles/Masters+25/Gedanken...,Gedankenwirbel,Masters+25,Common,4101
2018-03-28,0.54,/de/Magic/Products/Singles/Masters+25/Gedanken...,Gedankenwirbel,Masters+25,Common,4101
2018-03-29,0.48,/de/Magic/Products/Singles/Masters+25/Gedanken...,Gedankenwirbel,Masters+25,Common,4101
2018-03-30,0.63,/de/Magic/Products/Singles/Masters+25/Gedanken...,Gedankenwirbel,Masters+25,Common,4101
